In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
import ast
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)

# 1 Conceptual issues for LTCM
## 1.1 Describe LTCM’s investment strategy with regard to the following aspects:

- Securities traded: Fixed income and credit, and some equities, acoree global markets. 

- Trading frequency: Their strategies are designed to make money over long horizons -- six months to two years or more, however the trading frequency is related to specific trading strategies. Their largest bucket, the convergence trades, typically take weeks and months to converge. Overall, they are something of a medium-term frequency. Largely, they are not trying to arbitrage intraday movements, nor are they making long-term directional bets.

- Skewness (Do they seek many small wins or a few big hits?): LTCM largely is trying to pick up small premia in the convergence trades, which limits the upside of any individual trade, and leaves it substantial downside, given all the leverage.

- Forecasting (What is behind their selection of trades?):  Forecasting convergence.

## 1.2 What are LTCM’s biggest advantages over its competitors?

- Efficient financing. LTCM got very favorable terms on all financing—sometimes even zero haircut! Typically had small, if any, outlay.
- Fund size. Have market power even in the large market of institutional wholesale.
- Liquidity. LTCM has in place many mechanisms to ensure liquidity.
- Long-term horizon. In financing and assessing trades, LTCM takes a relatively long-term view.
- Hedged. LTCM avoids taking too much default risk or explicit directional bets.


## 1.3 The case discusses four types of funding risk facing LTCM:

- collateral haircuts: Collateral haircuts. For most trades, LTCM obtains 100% financing on a fully collateralized basis. They use value at risk measures and stress test for modelling potential profits and losses.

- repo maturity: LTCM goes against the norm by entering into relatively long-maturity repo. While much of it is overnight, LTCM uses contracts that typically have maturity of 6-12 months. Furthermore, LTCM manages their aggregate repo maturity.

- equity redemption: The firm is highly levered, so equity funding risk is especially important. LTCM restricts redemptions of equity year by year. The restriction is particularly strong in that unredeemed money is re-locked.

- loan access

## 1.4 LTCM is largely in the business of selling liquidity and volatility. Describe how LTCM accounts for liquidity risk in their quantitative measurements.

## 1.5 Is leverage risk currently a concern for LTCM?

## 1.6 What is the risk in these convergence trades?

# 2 LTCM Risk Decomposition

## Data

In [2]:
ret = pd.read_excel("../data/ltcm_exhibits_data.xlsx", index_col=0, sheet_name=1)
ret.head()

,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
NaT,NaN,NaN,NaN,1.00
1994-03-01,1.1,-0.011,-0.013,0.99
1994-04-01,1.1,0.014,0.008,1.00
1994-05-01,1.2,0.068,0.053,1.05
1994-06-01,1.2,-0.039,-0.029,1.02


In [3]:
SPY = pd.read_excel("../data/gmo_analysis_data.xlsx", sheet_name=2, index_col=0)
rf = pd.read_excel("../data/gmo_analysis_data.xlsx", sheet_name=3, index_col=0)
SPY = SPY[["SPY"]]
SPY.head()

,SPY
1993-02-28,0.0107
1993-03-31,0.0224
1993-04-30,-0.0256
1993-05-31,0.0270
1993-06-30,0.0037


In [4]:
rf.head()

,US3M
1993-02-28,0.0025
1993-03-31,0.0025
1993-04-30,0.0025
1993-05-31,0.0026
1993-06-30,0.0026


## Functions

In [5]:
def summary_stat(df, annual_factor, q=0.05):
    '''summary assets' mean return, voaltility(stdev) and sharpe ratio'''
    result = pd.DataFrame()
    result["mean"] = df.mean() * annual_factor
    result["volatility"] = df.std() * np.sqrt(annual_factor)
    result["Sharpe Ratio"] = result["mean"]/result["volatility"]

    return result

def cal_risk(df, quant=0.05, style=True):
    result = pd.DataFrame()
    # you don't neet to annualize higher moments
    var_name = f'VaR ({quant})'
    cvar_name = f'CVaR ({quant})'
    result['skewness'] = df.skew()
    result['kurtosis'] = df.kurtosis()
    result[var_name] = df.quantile(quant)
    result[cvar_name] = df[df < df.quantile(quant)].mean()
    if style:
        result = result.style.format(
            {var_name: "{:.2%}", cvar_name: "{:.2%}"})
    return result

In [6]:
def get_capm_matrics(targets, regressors, add_constant=True, annualize_factor=12):

    result = pd.DataFrame(index=targets.columns)

    if add_constant:
        X = sm.add_constant(regressors)
    else:
        X = regressors.copy()
    for column in targets.columns:
        y = targets[[column]]
        model = sm.OLS(y, X, missing='drop').fit()
        if add_constant:
            result.loc[column, "alpha"] = model.params['const'] * \
                annualize_factor
        result.loc[column, regressors.columns] = model.params[regressors.columns]

        result.loc[column, "R-squared"] = model.rsquared

    return result

## 2.1 Summary stats.
### (a) For both the gross and net series of LTCM excess returns, report the mean, volatility, and Sharpe ratios.

In [7]:
ret = ret.resample('M').last()
ret_all = ret[["Gross Monthly Performancea",
                    "Net Monthly Performanceb"]].merge(SPY, left_index=True, right_index=True)
ret_excess = ret_all.subtract(rf['US3M'], axis=0).dropna()
ret_excess.head()

,Gross Monthly Performancea,Net Monthly Performanceb,SPY
1994-03-31,-0.0140,-0.0160,-0.0449
1994-04-30,0.0107,0.0047,0.0079
1994-05-31,0.0644,0.0494,0.0123
1994-06-30,-0.0425,-0.0326,-0.0264
1994-07-31,0.1123,0.0803,0.0287


In [8]:
summary_stat(ret_excess, 12)

,mean,volatility,Sharpe Ratio
Gross Monthly Performancea,0.2421,0.1362,1.7769
Net Monthly Performanceb,0.1554,0.1118,1.3901
SPY,0.1738,0.1123,1.5479


## (b) Report the skewness, kurtosis, and (historic) VaR(.05).

In [9]:
cal_risk(ret_excess, quant=0.05)

,skewness,kurtosis,VaR (0.05),CVaR (0.05)
Gross Monthly Performancea,-0.2877,1.5866,-3.04%,-7.30%
Net Monthly Performanceb,-0.8102,2.9269,-2.64%,-6.87%
SPY,-0.4335,-0.3620,-4.64%,-5.14%


## (c) Comment on how these stats compare to SPY and other assets we have seen. How much do they differ between gross and net?

- Sharpe is very high relative to what we usually see.
- Volatility is sized similarly to SPY, and minimum return is not too bad.
- The difference between Gross and Net is not too large. Gross return has higher mean and higher volatility, and also has a higher Sharpe ratio. Which means that if you take more risk, you will have a higher risk compensation per unit. 

## 2.2 Using the series of net LTCM excess returns, estimate the regresssion. 
### (a) Report $\alpha$ and $\beta_m$. Report the $R^2$ stat.

In [10]:
print("Annualized alpha: ")
get_capm_matrics(ret_excess[["Net Monthly Performanceb"]], ret_excess[["SPY"]], add_constant=True, annualize_factor=12)

Annualized alpha: 


,alpha,SPY,R-squared
Net Monthly Performanceb,0.1315,0.1371,0.019


### (b) From this regression, does LTCM appear to be a “closet indexer”?
- Not at all. Though LTCM delivers similar returns with resspect to SPY, it has very low correlation to SPY, and most of it's returns comes from alpha. 

### (c) From the regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?
- Yes. LTCM has a relatively low exposure to SPY and has an alpha which generating a massive information ratio. 

## 2.3 Let’s check for non-linear market exposure. Run the following regression on LTCM’s net excess returns
### (a) Report $\beta_1$, $\beta_2$, and the $R^2$ stat

In [11]:
ret_excess["rm^2"] = np.square(ret_excess["SPY"])

In [28]:
print("Annualized alpha: ")
get_capm_matrics(ret_excess[["Net Monthly Performanceb"]], ret_excess[[
                 "SPY", "rm^2"]], add_constant=True, annualize_factor=12)

Annualized alpha: 


,alpha,SPY,rm^2,R-squared
Net Monthly Performanceb,0.155,0.1669,-1.9267,0.0243


### (b) Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?

- It increases $R^2$ from 1.9% to 2.43%, but does not add to the $R^2$ to a significant degree. 

### (c) From the regression evidence, does LTCM’s market exposure behave as if it is long market options or short market options?

- $\beta_2$ equal to -1.93 which shows that LTCM short market options since the option price is convex to the market underlying asset. 

### (d) Should we describe LTCM as being positively or negatively exposed to market volatility?

- Negative. This indicates LTCM underperforms particularly large SPY movements (whether on the upside or downside).

## 2.4 Let’s try to pinpoint the nature of LTCM’s nonlinear exposure. Does it come more from exposure to up-markets or down-markets? Run the following regression on LTCM’s net excess returns:
### (a) Report β1, β2, and the R2 stat.

In [13]:
ret_excess["up"] = [max(rm - 0.03, 0) for rm in ret_excess["SPY"]]
ret_excess["down"] = [max(-0.03 - rm, 0) for rm in ret_excess["SPY"]]
ret_excess.head()

,Gross Monthly Performancea,Net Monthly Performanceb,SPY,rm^2,up,down
1994-03-31,-0.0140,-0.0160,-0.0449,2.0131e-03,0.0,0.0149
1994-04-30,0.0107,0.0047,0.0079,6.2473e-05,0.0,0.0000
1994-05-31,0.0644,0.0494,0.0123,1.5248e-04,0.0,0.0000
1994-06-30,-0.0425,-0.0326,-0.0264,6.9850e-04,0.0,0.0000
1994-07-31,0.1123,0.0803,0.0287,8.2184e-04,0.0,0.0000


In [14]:
print("Annualized alpha: ")
get_capm_matrics(ret_excess[["Net Monthly Performanceb"]], ret_excess[[
                 "SPY", "up", "down"]], add_constant=True, annualize_factor=12)

Annualized alpha: 


,alpha,SPY,up,down,R-squared
Net Monthly Performanceb,0.1012,0.4666,-0.7821,1.2896,0.0555


### (b) Is LTCM long or short the call-like factor? And the put-like factor?
- $\beta^{Up}$ is negative so LTCM appears short the call-like factor.
- $\beta^{Down}$ is positive so LTCM appears long the put-like factor.

### (c) Which factor moves LTCM more, the call-like factor, or the put-like factor?
- The absolute value of $\beta^{Down}$ is larger, so the call-like factor moves LTCM more. 

### (d) In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market’s upside? Short the market’s downside? Something else?
- LTCM short the call-like factor and long the put-like factor, and it has a negative exposure to marker volatility at all. So it must short the upside more than long the downside. 

# 3 The FX Carry Trade
## Data

In [15]:
rf_currency = pd.read_excel("../data/fx_carry_data.xlsx", index_col=0, sheet_name=1)
fx_rate = pd.read_excel("../data/fx_carry_data.xlsx",
                        index_col=0, sheet_name=2)
rf_currency.head()

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
DATE,,,,,
1999-01-31,0.0041,0.0049,0.0026,0.0010,3.3463e-04
1999-02-28,0.0041,0.0046,0.0026,0.0010,2.3229e-04
1999-03-31,0.0041,0.0044,0.0025,0.0010,1.4271e-04
1999-04-30,0.0041,0.0044,0.0021,0.0008,9.8958e-05
1999-05-31,0.0041,0.0044,0.0021,0.0008,7.5000e-05


In [16]:
fx_rate.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,1.6457,1.1371,0.7058,0.0086
1999-02-28,1.6027,1.0995,0.6899,0.0084
1999-03-31,1.6140,1.0808,0.6768,0.0084
1999-04-30,1.6085,1.0564,0.6554,0.0084
1999-05-31,1.6020,1.0422,0.6545,0.0083


In [17]:
log_rf = np.log(1 + rf_currency)
log_fx = np.log(fx_rate)

## Functions

In [18]:
def get_each_reg(targets, regressors, annualize_factor=12):

    result = pd.DataFrame(index=targets.columns)

    for i in range(targets.shape[1]):
        X = regressors.iloc[:, i]
        X = sm.add_constant(X)
        y = targets.iloc[:, i]
        column = targets.columns[i]
        model = sm.OLS(y, X, missing='drop').fit()
        result.loc[column, "alpha"] = model.params['const'] * annualize_factor
        result.loc[column, "beta"] = model.params[1]
        result.loc[column, "R-squared"] = model.rsquared

    return result.T

## 1 The Static Carry Trade

In [19]:
real_rf = log_fx.diff().values + log_rf.shift().values[:, 1:]
real_rf = pd.DataFrame(real_rf, index=log_fx.index, columns=log_fx.columns)
real_rf.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,NaN,NaN,NaN,NaN
1999-02-28,-0.0216,-0.0310,-0.0218,-0.0227
1999-03-31,0.0116,-0.0146,-0.0181,0.0025
1999-04-30,0.0010,-0.0203,-0.0311,-0.0083
1999-05-31,0.0003,-0.0114,-0.0007,-0.0120


In [20]:
excess_ret = log_fx.diff().values + log_rf.shift().values[:, 1:] - \
    np.repeat(log_rf[["USD1M"]].shift().values, 4, axis=1)

In [21]:
excess_ret = pd.DataFrame(
    excess_ret, index=log_fx.index, columns=log_fx.columns)
excess_ret.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,NaN,NaN,NaN,NaN
1999-02-28,-0.0257,-0.0351,-0.0259,-0.0268
1999-03-31,0.0075,-0.0187,-0.0222,-0.0016
1999-04-30,-0.0031,-0.0245,-0.0352,-0.0124
1999-05-31,-0.0037,-0.0155,-0.0048,-0.0160


In [22]:
summary_stat(excess_ret, 12)

,mean,volatility,Sharpe Ratio
USUK,-0.0035,0.0863,-0.0406
USEU,-0.0044,0.0947,-0.0459
USSZ,0.0043,0.0988,0.0437
USJP,-0.0174,0.0915,-0.1903


- Mean returns are tiny and tend to be negative for currency pairs apart from the US dollar and the Swiss franc. The only returns of notable magnitude stem from the USD and JPY trade.

## 3.2 Implications for UIP:
### (a) Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?

- UIP implies that the mean excess return of each currency relative to USD should be zero. However, the USD and JPY trade seems to have a mean excess return significantly different from zero. 
- The evidence from other currencies is not significent. Even the rate is near zero, we do not know wheather it is statistically significant.

### (b) A long position in which foreign currency offered the best Sharpe ratio over the sample?
- Being long CHF would have delivered the best Sharpe ratio over the sample.

### (c) Are there any foreign currencies for which a long position earned a negative excess return (in USD) over the sample?

- A long position in three of **GBP, EUR, JPY** currencies had negative mean excess (log) returns.

## 3.3 Predicting FX
### (a) Make a table with columns corresponding to a different currency regression. Report the regression estimates αi and βi in the first two rows. Report the R2 stat in the third row.

In [23]:
log_fx_diff = log_fx.diff().dropna()
rf_excess = np.repeat(log_rf[["USD1M"]].shift().values, 4, axis=1) - \
    log_rf.shift().values[:, 1:]
rf_excess = pd.DataFrame(
    rf_excess, index=log_fx.index, columns=log_fx.columns)
rf_excess = rf_excess.dropna()

In [24]:
reg_result = get_each_reg(log_fx_diff, rf_excess, annualize_factor=12)
reg_result

,USUK,USEU,USSZ,USJP
alpha,-0.0059,0.0070,0.0436,-0.0060
beta,0.4858,-1.2564,-1.6466,0.3715
R-squared,0.0004,0.0026,0.0039,0.0005


### (b) Suppose the foreign risk-free rate increases relative to the US rate.
For which foreign currencies would we predict a relative strengthening of the USD in the following period?

- **Answers:** GBP and JPY. They have beta bigger than 0. 

For which currencies would we predict relative weakening of the USD in the following period?

- **Answers:** EUR and CHF. They have beta smaller than 0. 

This FX predictability is strongest in the case of which foreign currency?

- **Answers:** CHF. It has the highest r-squared. 

## 3.4 The Dynamic Carry Trade
### (a) Calculate the fraction of months for which the estimated FX risk premium positive.

In [25]:
betas = reg_result.iloc[1, :]
excess_ret_expectation = reg_result.iloc[0, :]/12 + (betas - 1) * rf_excess
excess_ret_expectation

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-6.8394e-05,-0.0028,-0.0046,-0.0029
1999-03-31,-2.3875e-04,-0.0029,-0.0046,-0.0029
1999-04-30,-3.2543e-04,-0.0031,-0.0046,-0.0030
1999-05-31,-3.2588e-04,-0.0038,-0.0050,-0.0030
1999-06-30,-3.3511e-04,-0.0039,-0.0050,-0.0030
...,...,...,...,...
2021-06-30,-5.0427e-04,-0.0007,0.0017,-0.0006
2021-07-31,-5.0863e-04,-0.0007,0.0016,-0.0006
2021-08-31,-5.0551e-04,-0.0007,0.0017,-0.0006


In [26]:
month_frac = pd.DataFrame(
    data=None, columns=excess_ret_expectation.columns, index=['% of Months'])

for col in excess_ret_expectation.columns:
    month_frac[col] = (
        len(excess_ret_expectation[excess_ret_expectation[col] > 0])/len(excess_ret_expectation)) * 100

month_frac

,USUK,USEU,USSZ,USJP
% of Months,23.8095,50.1832,63.0037,0.0


### (b) Which currencies most consistently have a positive FX risk premium? And for which currencies does the FX risk premium most often go negative?

- **CHF** most consistantly has a positive FX risk premium. 
- **JPY** most often go negative.

### (c) Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.
We can use forecasts of excess returns to construct trading positions, (weights,) which vary with the forecasts. The resulting trading strategy should be better than the static carry trade.

That is, we could time the magnitude and direction of the currency trades instead of being passively 100\% invested in a currency according to the evidence in 3.3.
